#### The latitude & longitude in Beijing and Shanghai is topsy-turvy in their image name because of different sources
#### (Google does not have data in China.)
#### So when calculate the coordinate accuracy please switch lat & lon for Beijing and Shanghai

#### City Prediction Accuracy

In [118]:
import json
import csv

results_path = "/data1/ouyangtianjian/Gemini_GPT/results_final"
cities = ['Cape Town', 'London', 'Moscow', 'Mumbai', 'Paris', 'Sao Paulo', 'Sydney', 'Tokyo', 'New York', 'San Francisco', 'Nairobi', 'Beijing', 'Shanghai']
models = ['cogvlm2', 'llama3_llava_next_8b', 'llava34b', 'MiniCPM-V-2.5']
csv_file_path = "city_model_accuracy.csv"

# Initialize the results dictionary
results = {model: {city: 0 for city in cities} for model in models}

# Calculate accuracy for each city and model
for city in cities:
    for model in models:
        city_clean = city.replace(" ", "")
        try:
            with open(f"{results_path}/{city_clean}/{model}/img2cityloc.jsonl", "r") as f:
                img2cityloc = [json.loads(line) for line in f]

            # City accuracy calculation
            count = 0
            for data in img2cityloc:
                if city in data['text'] or city_clean in data['text'] or "S\u00e3o Paulo" in data['text'] or "S\u00e3oPaulo" in data['text']:
                    count += 1
            acc = count / len(img2cityloc) if img2cityloc else 0
            results[model][city] = acc
        except FileNotFoundError:
            results[model][city] = 'N/A'

# Write results to CSV
with open(csv_file_path, mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    header = ['Model'] + cities
    writer.writerow(header)
    
    for model, city_accuracies in results.items():
        row = [model] + [city_accuracies[city] for city in cities]
        writer.writerow(row)

print(f"Results have been written to {csv_file_path}")


Results have been written to city_model_accuracy.csv


#### Coordinate Prediction Accuracy

In [119]:
import json
import math
import json
import csv
import numpy as np

def getDist_points(q, v):
    try:
        EARTH_REDIUS = 6378.137
        v=np.array(v).reshape(-1,2)
        q=np.array(q).reshape(-1,2)
        v, q = np.deg2rad(v), np.deg2rad(q)
        
        lat1, lat2 = v[:,1].reshape(-1,1), q[:,1].reshape(-1,1)
        lng1, lng2 = v[:,0].reshape(-1,1), q[:,0].reshape(-1,1)
        sin_lats = np.dot(np.sin(lat1), np.sin(lat2.T))
        cos_lats = np.dot(np.cos(lat1), np.cos(lat2.T))
        cos_lng_diff = np.cos(lng2.reshape(-1) - lng1.reshape(-1,1))
        s = np.arccos(sin_lats + cos_lats*cos_lng_diff)
        d = s * EARTH_REDIUS ###* 1000
        d = d.T # q to v
        return d
    except:
        print('error')
    

#like above way to compute the distance
def get_two_point_distance(lat1, lon1, lat2, lon2):
    EARTH_REDIUS = 6378.137
    
    #deg2rad
    
    sin_lats = math.sin(math.radians(lat1)) * math.sin(math.radians(lat2))
    cos_lats = math.cos(math.radians(lat1)) * math.cos(math.radians(lat2))
    cos_lng_diff = math.cos(math.radians(lon2) - math.radians(lon1))
    s = math.acos(sin_lats + cos_lats * cos_lng_diff)
    d = s * EARTH_REDIUS
    return d

def compute_accuracy_planet(pred_coordinates_list, true_coordinates_list):
    accuracy_planet = {
        "1km": 0,
        "25km": 0,
    }
    for idx in range(len(pred_coordinates_list)):

        distance_km = getDist_points([pred_coordinates_list[idx][1], pred_coordinates_list[idx][0]], [true_coordinates_list[idx][1], true_coordinates_list[idx][0]])
        distance_km=distance_km[0][0]
        if distance_km <= 1:
            accuracy_planet["1km"] += 1
        if distance_km <= 25:
            accuracy_planet["25km"] += 1

    for k in accuracy_planet.keys():
        accuracy_planet[k] /= len(true_coordinates_list)
        accuracy_planet[k] *= 100.0 # percent

    return accuracy_planet

results_path = "/data1/ouyangtianjian/Gemini_GPT/results_final"

cities = ['Cape Town', 'London', 'Moscow', 'Mumbai', 'Paris', 'Sao Paulo', 'Sydney', 'Tokyo', 'New York', 'San Francisco', 'Nairobi', 'Beijing', 'Shanghai']
models = ['cogvlm2', 'llama3_llava_next_8b', 'llava34b', 'MiniCPM-V-2.5']

results = []

for City in cities:
    city_results = [City]
    city = City.replace(" ", "")
    for model in models:
        file_path = f"{results_path}/{city}/{model}/img2cityloc.jsonl"
        with open(file_path, "r") as f:
            img2cityloc = [json.loads(line) for line in f]

            true_lat = [float(data['img_name'].split("&")[1]) for data in img2cityloc]
            true_lng = [float(data['img_name'].split("&")[2]) for data in img2cityloc]
            true_coords = list(zip(true_lat, true_lng))

            pred_coords = []
            for data in img2cityloc:
                try:
                    pred = data['text'].split("(")[1].split(")")[0]
                    pred_lat = float(pred.split(", ")[0])
                    pred_lng = float(pred.split(", ")[1])
                    pred_coords.append((pred_lat, pred_lng))
                except:
                    pred_coords.append((0, 0))
        result = compute_accuracy_planet(pred_coords, true_coords)
        city_results.append(result)
    results.append(city_results)

csv_file_path = "/data1/ouyangtianjian/Gemini_GPT/results_final/coord_accuracy_results.csv"

with open(csv_file_path, "w", newline="") as csvfile:
    csv_writer = csv.writer(csvfile)
    header = ["City"] + models
    csv_writer.writerow(header)
    csv_writer.writerows(results)

print(f"Results have been saved to {csv_file_path}")  

Results have been saved to /data1/ouyangtianjian/Gemini_GPT/results_final/coord_accuracy_results.csv


#### GPT4o

In [ ]:
import json
import csv

results_path = "/data1/ouyangtianjian/Gemini_GPT/results_GPT4o"
cities = ['Cape Town', 'London', 'Moscow', 'Mumbai', 'Paris', 'Sao Paulo', 'Sydney', 'Tokyo', 'New York', 'San Francisco', 'Nairobi', 'Beijing', 'Shanghai']
models = ['GPT4o']
csv_file_path = "/data1/ouyangtianjian/Gemini_GPT/results_GPT4o/city_model_accuracy.csv"

# Initialize the results dictionary
results = {model: {city: 0 for city in cities} for model in models}

# Calculate accuracy for each city and model
for city in cities:
    for model in models:
        city_clean = city.replace(" ", "")
        try:
            with open(f"{results_path}/{city_clean}.jsonl", "r") as f:
                img2cityloc = [json.loads(line) for line in f]

            # City accuracy calculation
            count = 0
            for data in img2cityloc:
                if city in data['text'] or city_clean in data['text'] or "S\u00e3o Paulo" in data['text'] or "S\u00e3oPaulo" in data['text']:
                    count += 1
            acc = count / len(img2cityloc) if img2cityloc else 0
            results[model][city] = acc
        except FileNotFoundError:
            results[model][city] = 'N/A'

# Write results to CSV
with open(csv_file_path, mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    header = ['Model'] + cities
    writer.writerow(header)
    
    for model, city_accuracies in results.items():
        row = [model] + [city_accuracies[city] for city in cities]
        writer.writerow(row)

print(f"Results have been written to {csv_file_path}")


Results have been written to /data1/ouyangtianjian/Gemini_GPT/results_GPT4o/city_model_accuracy.csv


In [ ]:
import json

results_path = "/data1/ouyangtianjian/Gemini_GPT/results_GPT4o"

for City in ['Cape Town', 'London', 'Moscow', 'Mumbai', 'Paris', 'Sao Paulo', 'Sydney', 'Tokyo', 'New York', 'San Francisco', 'Nairobi', 'Beijing', 'Shanghai']:
    for model in ['GPT-4o']:
        city = City.replace(" ", "")
        with open(f"{results_path}/{city}.jsonl", "r") as f:
            img2cityloc = [json.loads(line) for line in f]

            true_lat = [float(data['img_name'].split("&")[1]) for data in img2cityloc]
            true_lng = [float(data['img_name'].split("&")[2]) for data in img2cityloc]
            true_coords = list(zip(true_lat, true_lng))

            pred_coords = []
            for data in img2cityloc:
                try: 
                    pred = data['text'].split("(")[1].split(")")[0]
                    pred_lat = float(pred.split(", ")[0])
                    pred_lng = float(pred.split(", ")[1])
                    pred_coords.append((pred_lat, pred_lng))
                except:
                    pred_coords.append((0, 0))
            print((true_coords))
            print((pred_coords))
        break
    break
            

[(-33.93072696202203, 18.59051504062446), (-33.91616264501153, 18.46453535678856), (-33.87165782440083, 18.61682761759354), (-33.92862177164808, 18.6913128206804), (-34.03693523532098, 18.4807356641909), (-33.90757598591173, 18.47486561697253), (-34.02260804156846, 18.6812348868726), (-34.03200105908059, 18.4994010301657), (-33.83247450699062, 18.54871812776414), (-34.06107119748123, 18.61853231512634), (-34.01751555353447, 18.36431170770186), (-33.86530277378198, 18.55746592431864), (-33.85767362870086, 18.64736696046198), (-33.83195399045024, 18.50200510916642), (-33.82578880156112, 18.64778240957012), (-33.89360621469505, 18.62114550549953), (-33.98780591150167, 18.55244375993837), (-33.90466130265246, 18.5880691289398), (-33.82067638105513, 18.67848443778809), (-34.02941774063159, 18.42823952223427), (-33.9817897913373, 18.36263604281537), (-33.99877538056389, 18.43623576772829), (-33.92711843749734, 18.63198081352246), (-33.95475285111034, 18.38106800036384), (-33.90575445184358, 

In [ ]:
import json
import math
import webdataset as wds
import s2sphere
import tqdm
import re
import numpy as np

def getDist_points(q, v):
    try:
        EARTH_REDIUS = 6378.137
        v=np.array(v).reshape(-1,2)
        q=np.array(q).reshape(-1,2)
        v, q = np.deg2rad(v), np.deg2rad(q)
        
        lat1, lat2 = v[:,1].reshape(-1,1), q[:,1].reshape(-1,1)
        lng1, lng2 = v[:,0].reshape(-1,1), q[:,0].reshape(-1,1)
        sin_lats = np.dot(np.sin(lat1), np.sin(lat2.T))
        cos_lats = np.dot(np.cos(lat1), np.cos(lat2.T))
        cos_lng_diff = np.cos(lng2.reshape(-1) - lng1.reshape(-1,1))
        s = np.arccos(sin_lats + cos_lats*cos_lng_diff)
        d = s * EARTH_REDIUS ###* 1000
        d = d.T # q to v
        return d
    except:
        print('error')
    

def compute_accuracy_planet(pred_coordinates_list, true_coordinates_list):
    accuracy_planet = {
        "1km": 0,
        "25km": 0,
    }
    for idx in range(len(pred_coordinates_list)):

        distance_km = getDist_points([pred_coordinates_list[idx][1], pred_coordinates_list[idx][0]], [true_coordinates_list[idx][1], true_coordinates_list[idx][0]])
        distance_km=distance_km[0][0]
        if distance_km <= 1:
            accuracy_planet["1km"] += 1
        if distance_km <= 25:
            accuracy_planet["25km"] += 1
        # if distance_km <= 200:
        #     accuracy_planet["region"] += 1
        # if distance_km <= 750:
        #     accuracy_planet["country"] += 1
        # if distance_km <= 2500:
        #     accuracy_planet["continent"] += 1

    for k in accuracy_planet.keys():
        accuracy_planet[k] /= len(true_coordinates_list)
        accuracy_planet[k] *= 100.0 # percent

    return accuracy_planet

results_path = "/data1/ouyangtianjian/Gemini_GPT/results_GPT4o"

import json
import csv
import os

cities = ['Cape Town', 'London', 'Moscow', 'Mumbai', 'Paris', 'Sao Paulo', 'Sydney', 'Tokyo', 'New York', 'San Francisco', 'Nairobi', 'Beijing', 'Shanghai']
models = ['GPT-4o']

results = []

for City in cities:
    city_results = [City]
    city = City.replace(" ", "")
    for model in models:
        file_path = f"{results_path}/{city}.jsonl"
        with open(file_path, "r") as f:
            img2cityloc = [json.loads(line) for line in f]

            true_lat = [float(data['img_name'].split("&")[1]) for data in img2cityloc]
            true_lng = [float(data['img_name'].split("&")[2]) for data in img2cityloc]
            true_coords = list(zip(true_lat, true_lng))

            pred_coords = []
            for data in img2cityloc:
                try:
                    pred = data['text'].split("(")[1].split(")")[0]
                    pred_lat = float(pred.split(", ")[0])
                    pred_lng = float(pred.split(", ")[1])
                    pred_coords.append((pred_lat, pred_lng))
                except:
                    pred_coords.append((0, 0))
        result = compute_accuracy_planet(pred_coords, true_coords)
        city_results.append(result)
    results.append(city_results)

csv_file_path = "/data1/ouyangtianjian/Gemini_GPT/results_GPT4o/coord_accuracy_results.csv"

with open(csv_file_path, "w", newline="") as csvfile:
    csv_writer = csv.writer(csvfile)
    header = ["City"] + models
    csv_writer.writerow(header)
    csv_writer.writerows(results)

print(f"Results have been saved to {csv_file_path}") 

Results have been saved to /data1/ouyangtianjian/Gemini_GPT/results_GPT4o/coord_accuracy_results.csv


#### Qwen-xl-plus

In [116]:
results_path = "/data1/ouyangtianjian/Gemini_GPT/results_Qwen"

for City in ['Cape Town', 'London', 'Moscow', 'Mumbai', 'Paris', 'Sao Paulo', 'Sydney', 'Tokyo', 'New York', 'San Francisco', 'Nairobi', 'Beijing', 'Shanghai']:
    city = City.replace(" ", "")
    with open(f"{results_path}/{city}.txt", "r") as f:
        lines = f.readlines()

    count = 0
    for line in lines:
        answer = line.split(": ")[1]
        pred_city = answer.split(". (")[0]
        if City in pred_city or city in pred_city or "S\u00e3o Paulo" in pred_city or "S\u00e3oPaulo" in pred_city or "São Paulo" in pred_city or "SãoPaulo" in pred_city:
            count += 1

    acc = count / len(lines)
    print(f"{City}: {acc}")     

Cape Town: 0.962
London: 0.988
Moscow: 0.826
Mumbai: 0.802
Paris: 0.936
Sao Paulo: 0.322
Sydney: 0.892
Tokyo: 1.0
New York: 0.896
San Francisco: 0.712
Nairobi: 0.998
Beijing: 0.93
Shanghai: 0.045816733067729085


In [95]:
results_path = "/data1/ouyangtianjian/Gemini_GPT/results_Qwen"

for City in ['Cape Town', 'London', 'Moscow', 'Mumbai', 'Paris', 'Sao Paulo', 'Sydney', 'Tokyo', 'New York', 'San Francisco', 'Nairobi', 'Beijing', 'Shanghai']:
    city = City.replace(" ", "")
    with open(f"{results_path}/{city}.txt", "r") as f:
        lines = f.readlines()
    
    true_coords = []
    pred_coords = []   

    for line in lines:
        try:
            true_lat = float(line.split("&")[2])
            true_lng = float(line.split("&")[1])
            true_coords.append((true_lat, true_lng))
        except:
            true_coords.append((0, 0))

        answer = line.split(": ")[1]
        try:
            coords = answer.split(". (")[1].split(")")[0]
            pred_lat = float(coords.split(", ")[0])
            pred_lng = float(coords.split(", ")[1])
            pred_coords.append((pred_lat, pred_lng))
        except:
            pred_coords.append((0, 0))
        
    result = compute_accuracy_planet(pred_coords, true_coords)
    print(f"{City}: {result}")
    # print(true_coords)
    # print(pred_coords)

Cape Town: {'1km': 0.0, '25km': 0.0}
London: {'1km': 0.0, '25km': 0.0}
Moscow: {'1km': 0.0, '25km': 0.0}
Mumbai: {'1km': 0.0, '25km': 0.0}
Paris: {'1km': 0.0, '25km': 0.0}
Sao Paulo: {'1km': 0.0, '25km': 0.0}
Sydney: {'1km': 0.0, '25km': 0.0}
Tokyo: {'1km': 0.0, '25km': 0.0}
New York: {'1km': 0.0, '25km': 0.2}
San Francisco: {'1km': 0.0, '25km': 0.2}
Nairobi: {'1km': 0.0, '25km': 0.0}
Beijing: {'1km': 0.2, '25km': 79.2}
Shanghai: {'1km': 0.398406374501992, '25km': 0.796812749003984}


#### Internvl

In [ ]:
import json
import csv

results_path = "/data1/ouyangtianjian/llm_benchmark/code/InternVL/results"
cities = ['Cape Town', 'London', 'Moscow', 'Mumbai', 'Paris', 'Sao Paulo', 'Sydney', 'Tokyo', 'New York', 'San Francisco', 'Nairobi', 'Beijing', 'Shanghai']
models = ['InternVL']
csv_file_path = "/data1/ouyangtianjian/llm_benchmark/code/InternVL/results/city_model_accuracy.csv"

# Initialize the results dictionary
results = {model: {city: 0 for city in cities} for model in models}

# Calculate accuracy for each city and model
for city in cities:
    for model in models:
        city_clean = city.replace(" ", "")
        try:
            with open(f"{results_path}/{city_clean}/InternVL-Chat-v1-5/img2cityloc_sv.jsonl", "r") as f:
                img2cityloc = [json.loads(line) for line in f]

            # City accuracy calculation
            count = 0
            for data in img2cityloc:
                if city in data['text'] or city_clean in data['text'] or "S\u00e3o Paulo" in data['text'] or "S\u00e3oPaulo" in data['text']:
                    count += 1
            acc = count / len(img2cityloc) if img2cityloc else 0
            results[model][city] = acc
        except FileNotFoundError:
            results[model][city] = 'N/A'

# Write results to CSV
with open(csv_file_path, mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    header = ['Model'] + cities
    writer.writerow(header)
    
    for model, city_accuracies in results.items():
        row = [model] + [city_accuracies[city] for city in cities]
        writer.writerow(row)

print(f"Results have been written to {csv_file_path}")


Results have been written to /data1/ouyangtianjian/llm_benchmark/code/InternVL/results/city_model_accuracy.csv


In [ ]:
import json
import math
import webdataset as wds
import s2sphere
import tqdm
import re
import numpy as np

def getDist_points(q, v):
    try:
        EARTH_REDIUS = 6378.137
        v=np.array(v).reshape(-1,2)
        q=np.array(q).reshape(-1,2)
        v, q = np.deg2rad(v), np.deg2rad(q)
        
        lat1, lat2 = v[:,1].reshape(-1,1), q[:,1].reshape(-1,1)
        lng1, lng2 = v[:,0].reshape(-1,1), q[:,0].reshape(-1,1)
        sin_lats = np.dot(np.sin(lat1), np.sin(lat2.T))
        cos_lats = np.dot(np.cos(lat1), np.cos(lat2.T))
        cos_lng_diff = np.cos(lng2.reshape(-1) - lng1.reshape(-1,1))
        s = np.arccos(sin_lats + cos_lats*cos_lng_diff)
        d = s * EARTH_REDIUS ###* 1000
        d = d.T # q to v
        return d
    except:
        print('error')
    

#like above way to compute the distance
def get_two_point_distance(lat1, lon1, lat2, lon2):
    EARTH_REDIUS = 6378.137
    
    #deg2rad
    
    sin_lats = math.sin(math.radians(lat1)) * math.sin(math.radians(lat2))
    cos_lats = math.cos(math.radians(lat1)) * math.cos(math.radians(lat2))
    cos_lng_diff = math.cos(math.radians(lon2) - math.radians(lon1))
    s = math.acos(sin_lats + cos_lats * cos_lng_diff)
    d = s * EARTH_REDIUS
    return d

def compute_accuracy_planet(pred_coordinates_list, true_coordinates_list):
    accuracy_planet = {
        "1km": 0,
        "25km": 0,
    }
    for idx in range(len(pred_coordinates_list)):

        distance_km = getDist_points([pred_coordinates_list[idx][1], pred_coordinates_list[idx][0]], [true_coordinates_list[idx][1], true_coordinates_list[idx][0]])
        distance_km=distance_km[0][0]
        if distance_km <= 1:
            accuracy_planet["1km"] += 1
        if distance_km <= 25:
            accuracy_planet["25km"] += 1
        # if distance_km <= 200:
        #     accuracy_planet["region"] += 1
        # if distance_km <= 750:
        #     accuracy_planet["country"] += 1
        # if distance_km <= 2500:
        #     accuracy_planet["continent"] += 1

    for k in accuracy_planet.keys():
        accuracy_planet[k] /= len(true_coordinates_list)
        accuracy_planet[k] *= 100.0 # percent

    return accuracy_planet

results_path = "/data1/ouyangtianjian/llm_benchmark/code/InternVL/results"

import json
import csv
import os

cities = ['Cape Town', 'London', 'Moscow', 'Mumbai', 'Paris', 'Sao Paulo', 'Sydney', 'Tokyo', 'New York', 'San Francisco', 'Nairobi', 'Beijing', 'Shanghai']
models = ['InternVL']

results = []

for City in cities:
    city_results = [City]
    city = City.replace(" ", "")
    for model in models:
        file_path = f"{results_path}/{city}/InternVL-Chat-v1-5/img2cityloc_sv.jsonl"
        with open(file_path, "r") as f:
            img2cityloc = [json.loads(line) for line in f]

            true_lat = [float(data['img_name'].split("&")[1]) for data in img2cityloc]
            true_lng = [float(data['img_name'].split("&")[2]) for data in img2cityloc]
            true_coords = list(zip(true_lat, true_lng))

            pred_coords = []
            for data in img2cityloc:
                try:
                    pred = data['text'].split("(")[1].split(")")[0]
                    pred_lat = float(pred.split(", ")[1])
                    pred_lng = float(pred.split(", ")[0])
                    pred_coords.append((pred_lat, pred_lng))
                except:
                    pred_coords.append((0, 0))
            print(pred_coords)
        result = compute_accuracy_planet(pred_coords, true_coords)
        city_results.append(result)
    results.append(city_results)

csv_file_path = "/data1/ouyangtianjian/llm_benchmark/code/InternVL/results/coord_accuracy_results.csv"

with open(csv_file_path, "w", newline="") as csvfile:
    csv_writer = csv.writer(csvfile)
    header = ["City"] + models
    csv_writer.writerow(header)
    csv_writer.writerows(results)

print(f"Results have been saved to {csv_file_path}") 

[(18.4241, 33.9249), (0, 0), (-80.1918, 25.7617), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (18.4241, 33.9249), (0, 0), (0, 0), (0, 0), (18.4241, 33.9249), (0, 0), (-118.324755, 34.148331), (0, 0), (0, 0), (0, 0), (144.9631, 37.8136), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (18.4241, 33.9249), (18.4241, 33.9249), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (18.4241, 33.9249), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (18.4241, 33.9249), (0, 0), (18.4241, 33.9249), (0, 0), (18.4241, 33.9249), (0, 0), (0, 0), (18.4241, 33.9249), (0, 0), (0, 0), (-43.17292038, 22.9068487), (0, 0), (18.4241, 33.9249), (18.4241, 33.9249), (0, 0), (0, 0), (151.209311, 33.86882), (18.4241, 33.9249), (18.4241, 33.9249), (18.4241, 33.9249), (151.209311, 33.86882), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (151.209311, 33.86882), (0, 0), (0, 0), (0, 0), (0, 0), (18.4241, 33.9249), (0, 0), (151.209311, 33.86882), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (18.4241, 33.9249), (0, 0), (0, 0), (0,